In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [2]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,28117743919,14795251404,554516117,554516117,283582751,283582751,309281138,309281138,510198997,510198997
2,20770874747,14861950737,356651609,356651609,171192622,171192622,184772992,184772992,434882514,434882514
3,23606236854,15157164898,696147389,696147389,359746429,359746429,327626158,327626158,494508402,494508402
4,15730217253,14679307596,467684314,467684314,190005210,190005210,184524533,184524533,383508402,383508402


In [3]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [4]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [6]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [7]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 16s 8ms/step - loss: 0.0075 - val_loss: 0.0042
Epoch 2/20
2000/2000 [==============================] - 13s 7ms/step - loss: 0.0044 - val_loss: 0.0048
Epoch 3/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 4/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0032 - val_loss: 0.0037
Epoch 5/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0030 - val_loss: 0.0015
Epoch 6/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0028 - val_loss: 0.0044
Epoch 7/20
2000/2000 [==============================] - 13s 7ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 8/20
2000/2000 [==============================] - 13s 7ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 9/20
2000/2000 [==============================] - 13s 7ms/step - loss: 0.0024 - val_loss: 0.0030
Epoch 10/20
2000/2000 [=====

In [8]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[28117743919,554516117,283582751,309281138,510198997]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


[0.5704816  0.00588964 0.01030665 0.01159854 0.01008111]
[0.57038559 0.00513615 0.00749913 0.00811517 0.007527  ]
[0.65365404 0.02437584 0.01922711 0.01718993 0.02663473]
[0.73396538 0.02338422 0.00950026 0.00922623 0.01917542]


In [9]:
# Save trained model
model.save("model.h5")
